# Spaceships picker

Pick spaceships with same (or similar) behavior characteristics, randomly.

## Imports

In [ ]:
import numpy as np
import os

from typing import List, Tuple

## MAP-Elites properties

In [ ]:
BIN_N = (32, 32)
BIN_BOUNDS = (1, 1)

## Solutions parsing

Solutions are high-level spaceships strings saved in a `.log` file as

```
{spaceship_string}
{[fitness_1, fitness_2, ..., fitness_n]}
\r
```

In [ ]:
main_folder = 'results'

methods_names = ['FI-2Pop', 'm-FI-2Pop', 'CMAP-Elites', 'Em-CMAP-Elites']
folder_names = ['cog_experiment01', 'cog_experiment01', 'cog_experiment02', 'cog_experiment05']
log_names = ['standard-fi2pop-elites_atoms.log', 'variant-fi2pop-min_merge-elites_atoms.log', 'standard-mapelites-elites_atoms.log', 'variant-optim-mapelites-min_merge-elites_atoms.log']

def read_file(fname: str) -> Tuple[List[str], List[np.ndarray]]:
    strings, fitnesses = [], []
    
    with open(fname, 'r') as f:
        contents = f.readlines()

        i = 0
        while i < len(contents):
            strings.append(contents[i])
            fitnesses.append(np.asanyarray(list(map(float, contents[i+1].replace('Fitness: ', '').replace('[', '').replace(']', '').replace('\n', '').split(',')))))
            i += 3
    
    return strings, fitnesses

Parse all `log` files:

In [ ]:
all_strings = {}
all_fitnesses = {}

for method, logname, foldername in zip(methods_names, log_names, folder_names):
    fname = os.path.join(main_folder, foldername)
    logloc = os.path.join(fname, logname)
    ss, fs = read_file(fname=logloc)
    all_strings[method] = ss
    all_fitnesses[method] = fs

## Bins selection

In [ ]:
def to_grid_bin_idxs(fitness: np.ndarray) -> Tuple[int, int]:
    b1 = np.arange(0, BIN_BOUNDS[0], BIN_BOUNDS[0] / BIN_N[0])
    b2 = np.arange(0, BIN_BOUNDS[1], BIN_BOUNDS[1] / BIN_N[1])    
    bx = np.digitize(x=[fitness[0]], bins=b1, right=False)[0] - 1
    by = np.digitize(x=[fitness[1]], bins=b2, right=False)[0] - 1
    return (bx, by)

def pick_random_viable_index(ref_bc: Tuple[int, int],
                             fitnesses: List[np.ndarray]) -> int:
    bcs = [to_grid_bin_idxs(fitness=f) for f in fitnesses]
    if ref_bc in bcs:
        print(f'Found same BC: {ref_bc}')
        return bcs.index(ref_bc)
    else:
        scores = [np.abs(ref_bc[0] - b1) + np.abs(ref_bc[0] - b2) for (b1, b2) in bcs]
        closest = np.argmin(scores)
        print(f'Selecting closest BC: {bcs[closest]}')
        return closest

## Spaceships selection

In [ ]:
# Enforce selecting a specific bin by setting the value of ref_bc
ref_bc = None
# ref_bc = (28, 3)

In [ ]:
chosen_spaceships = {}
# pick a random spaceship for the first experiment
if not ref_bc:
    ref_spaceship_idx = np.random.choice(np.arange(len(all_strings[list(all_strings.keys())[0]])))
    ref_bc = to_grid_bin_idxs(fitness=all_fitnesses[list(all_fitnesses.keys())[0]][ref_spaceship_idx])
for k in all_strings.keys():
    print(f'Choosing spaceship for {k}...')
    i = pick_random_viable_index(ref_bc=ref_bc,
                                 fitnesses=all_fitnesses[k])
    print(f'Chosen spaceship is {all_strings[k][i]}')
